In [24]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [25]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Activation, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import math

# import scikitplot
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


from keras.utils import np_utils

### Loading the image model

In [26]:
# loading the model
with open('D:\sem_4_project_material\cnnsentiment_model2.json', 'r') as json_file:
    json_savedModel= json_file.read()

#load the model architecture 
model_j = tf.keras.models.model_from_json(json_savedModel)

In [27]:
model_j.load_weights('D:\sem_4_project_material\cnnsentiment_weights2.h5')

In [28]:
#Compiling the model
optims = [
    optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    optimizers.Adam(0.001),
]
model_j.compile(loss='binary_crossentropy',
         optimizer=optims[1],
         metrics=['accuracy'])

### creating test dataset

In [45]:
test_path = "D:/sem_4_project_material/test1/"

In [46]:
img_list = []
img_label = []
t = 0
Emotions = ['negative','positive']

for emotion in Emotions:
    for img in os.listdir(os.path.join(test_path,emotion)):
        name = os.path.basename(os.path.join(test_path,emotion + "/" + img))
        img_list.append(name)
        img_label.append(t)
        
    t += 1

In [47]:
img_df = pd.DataFrame({"image":img_list,"target":img_label})

In [48]:
img_df

,image,target
0,PrivateTest_1054527.jpg,0
1,PrivateTest_1109992.jpg,0
2,PrivateTest_1161501.jpg,0
3,PrivateTest_1221822.jpg,0
4,PrivateTest_1256234.jpg,0
...,...,...
162,S130_013_00000015 (copy).png,1
163,S133_009_00000004 (copy).png,1
164,S134_004_00000013 (copy).png,1
165,S136_001_00000018 (copy).png,1


In [30]:
data = pd.read_csv("D:/sem_4_project_material/cleaned_text.csv")
data

,target,clean_text
0,0,awww that has a bummer you shoulda got dav...
1,0,is upset that he cannot update his facebook by...
2,0,i dived many times for the ball managed to sa...
3,0,my whole body feels itchy and like its on fire
4,0,no it has not behaving at all I am mad why am...
...,...,...
1048570,1,back home thought I had done for the week but ...
1048571,1,my grandma is making dinenr with my mum
1048572,1,midmorning snack time a bowl of cheese noodles...
1048573,1,same here say it like from the terminiator m...


In [31]:
X = data["clean_text"]
y = data["target"]

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
X_train,X_cv,y_train,y_cv=train_test_split(X_train,y_train,test_size=0.3)

In [34]:
count_vec=CountVectorizer(ngram_range=(1,2),max_features=15000)
x_train_biagram=count_vec.fit_transform(X_train)
x_cv_biagram=count_vec.transform(X_cv)
x_test_biagram=count_vec.transform(X_test)

In [35]:
import pickle

In [36]:
with open("D:/sem_4_project_material/sentiment_model.pkl","rb") as f:
    model=pickle.load(f)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.0 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [41]:
pos_text = data[data['target']==1][:83]
pos_text.reset_index(inplace=True, drop=True)

In [42]:
pos_text['sn'] = pos_text.index

In [43]:
pos_text

,target,clean_text,sn
0,1,i love u guys r the best,0
1,1,im meeting up with one of my besties tonight c...,1
2,1,thanks for the twitter add sunisa i got to me...,2
3,1,being sick can be really cheap when it hurts t...,3
4,1,he has that effect on everyone,4
...,...,...,...
78,1,had a good tech meeting at clubzone dinner wa...,78
79,1,thanks for the hook up with and suggesting m...,79
80,1,depends on which version they thought you th...,80
81,1,hand is still pretty weak i cant like punch a...,81


In [49]:
pos_img = img_df[img_df['target']==1]
pos_img
# neg1['sn'] = neg1.index

,image,target
84,1.png,1
85,10.png,1
86,11.png,1
87,12.png,1
88,13.png,1
...,...,...
162,S130_013_00000015 (copy).png,1
163,S133_009_00000004 (copy).png,1
164,S134_004_00000013 (copy).png,1
165,S136_001_00000018 (copy).png,1


In [50]:
pos_img.reset_index(inplace=True, drop=True)
pos_img

,image,target
0,1.png,1
1,10.png,1
2,11.png,1
3,12.png,1
4,13.png,1
...,...,...
78,S130_013_00000015 (copy).png,1
79,S133_009_00000004 (copy).png,1
80,S134_004_00000013 (copy).png,1
81,S136_001_00000018 (copy).png,1


In [51]:
pos_img['sn'] = pos_img.index
pos_img

<ipython-input-51-eaa88a449bbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_img['sn'] = pos_img.index


,image,target,sn
0,1.png,1,0
1,10.png,1,1
2,11.png,1,2
3,12.png,1,3
4,13.png,1,4
...,...,...,...
78,S130_013_00000015 (copy).png,1,78
79,S133_009_00000004 (copy).png,1,79
80,S134_004_00000013 (copy).png,1,80
81,S136_001_00000018 (copy).png,1,81


In [52]:
# using x_test target column and image target column join the dataset
temp_df1 = pd.merge(pos_text,pos_img,on='sn')
temp_df1

,target_x,clean_text,sn,image,target_y
0,1,i love u guys r the best,0,1.png,1
1,1,im meeting up with one of my besties tonight c...,1,10.png,1
2,1,thanks for the twitter add sunisa i got to me...,2,11.png,1
3,1,being sick can be really cheap when it hurts t...,3,12.png,1
4,1,he has that effect on everyone,4,13.png,1
...,...,...,...,...,...
78,1,had a good tech meeting at clubzone dinner wa...,78,S130_013_00000015 (copy).png,1
79,1,thanks for the hook up with and suggesting m...,79,S133_009_00000004 (copy).png,1
80,1,depends on which version they thought you th...,80,S134_004_00000013 (copy).png,1
81,1,hand is still pretty weak i cant like punch a...,81,S136_001_00000018 (copy).png,1


In [53]:
temp_df1.drop(['target_x','sn'], axis=1,inplace=True)
temp_df1

,clean_text,image,target_y
0,i love u guys r the best,1.png,1
1,im meeting up with one of my besties tonight c...,10.png,1
2,thanks for the twitter add sunisa i got to me...,11.png,1
3,being sick can be really cheap when it hurts t...,12.png,1
4,he has that effect on everyone,13.png,1
...,...,...,...
78,had a good tech meeting at clubzone dinner wa...,S130_013_00000015 (copy).png,1
79,thanks for the hook up with and suggesting m...,S133_009_00000004 (copy).png,1
80,depends on which version they thought you th...,S134_004_00000013 (copy).png,1
81,hand is still pretty weak i cant like punch a...,S136_001_00000018 (copy).png,1


In [54]:
neg = data[data['target']==0][:84]

In [55]:
neg['sn'] = neg.index

In [56]:
neg

,target,clean_text,sn
0,0,awww that has a bummer you shoulda got dav...,0
1,0,is upset that he cannot update his facebook by...,1
2,0,i dived many times for the ball managed to sa...,2
3,0,my whole body feels itchy and like its on fire,3
4,0,no it has not behaving at all I am mad why am...,4
...,...,...,...
79,0,wonders why someone that u like so much can ma...,79
80,0,sleep soon i just hate saying bye and see you ...,80
81,0,just got ur newsletter those fares really are...,81
82,0,missin the boo,82


In [57]:
neg1 = img_df[img_df['target']==0]
neg1['sn'] = neg1.index

<ipython-input-57-88bc77c763b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg1['sn'] = neg1.index


In [58]:
neg1

,image,target,sn
0,PrivateTest_1054527.jpg,0,0
1,PrivateTest_1109992.jpg,0,1
2,PrivateTest_1161501.jpg,0,2
3,PrivateTest_1221822.jpg,0,3
4,PrivateTest_1256234.jpg,0,4
...,...,...,...
79,S134_008_00000011.png,0,79
80,S502_001_00000015.png,0,80
81,S504_001_00000020.png,0,81
82,S505_006_00000019.png,0,82


In [59]:
# using x_test target column and image target column join the dataset
temp_df2 = pd.merge(neg,neg1,on='sn')
temp_df2

,target_x,clean_text,sn,image,target_y
0,0,awww that has a bummer you shoulda got dav...,0,PrivateTest_1054527.jpg,0
1,0,is upset that he cannot update his facebook by...,1,PrivateTest_1109992.jpg,0
2,0,i dived many times for the ball managed to sa...,2,PrivateTest_1161501.jpg,0
3,0,my whole body feels itchy and like its on fire,3,PrivateTest_1221822.jpg,0
4,0,no it has not behaving at all I am mad why am...,4,PrivateTest_1256234.jpg,0
...,...,...,...,...,...
79,0,wonders why someone that u like so much can ma...,79,S134_008_00000011.png,0
80,0,sleep soon i just hate saying bye and see you ...,80,S502_001_00000015.png,0
81,0,just got ur newsletter those fares really are...,81,S504_001_00000020.png,0
82,0,missin the boo,82,S505_006_00000019.png,0


In [60]:
temp_df2.drop(['target_x','sn'],axis=1,inplace=True)
temp_df2

,clean_text,image,target_y
0,awww that has a bummer you shoulda got dav...,PrivateTest_1054527.jpg,0
1,is upset that he cannot update his facebook by...,PrivateTest_1109992.jpg,0
2,i dived many times for the ball managed to sa...,PrivateTest_1161501.jpg,0
3,my whole body feels itchy and like its on fire,PrivateTest_1221822.jpg,0
4,no it has not behaving at all I am mad why am...,PrivateTest_1256234.jpg,0
...,...,...,...
79,wonders why someone that u like so much can ma...,S134_008_00000011.png,0
80,sleep soon i just hate saying bye and see you ...,S502_001_00000015.png,0
81,just got ur newsletter those fares really are...,S504_001_00000020.png,0
82,missin the boo,S505_006_00000019.png,0


In [61]:
test_df = pd.concat([temp_df1,temp_df2])
test_df

,clean_text,image,target_y
0,i love u guys r the best,1.png,1
1,im meeting up with one of my besties tonight c...,10.png,1
2,thanks for the twitter add sunisa i got to me...,11.png,1
3,being sick can be really cheap when it hurts t...,12.png,1
4,he has that effect on everyone,13.png,1
...,...,...,...
79,wonders why someone that u like so much can ma...,S134_008_00000011.png,0
80,sleep soon i just hate saying bye and see you ...,S502_001_00000015.png,0
81,just got ur newsletter those fares really are...,S504_001_00000020.png,0
82,missin the boo,S505_006_00000019.png,0


In [62]:
test_df.reset_index(inplace=True, drop=True)
test_df

,clean_text,image,target_y
0,i love u guys r the best,1.png,1
1,im meeting up with one of my besties tonight c...,10.png,1
2,thanks for the twitter add sunisa i got to me...,11.png,1
3,being sick can be really cheap when it hurts t...,12.png,1
4,he has that effect on everyone,13.png,1
...,...,...,...
162,wonders why someone that u like so much can ma...,S134_008_00000011.png,0
163,sleep soon i just hate saying bye and see you ...,S502_001_00000015.png,0
164,just got ur newsletter those fares really are...,S504_001_00000020.png,0
165,missin the boo,S505_006_00000019.png,0


### function to predict the test dataset

In [1]:
img = cv2.imread( test_path + "positive/" + "1.png")
img = img/255.0
img = cv2.resize(img, (48,48))
img = img.reshape(1,48,48,3)
pred = list(model_j.predict(img)[0])

NameError: name 'cv2' is not defined

In [92]:
def predict(df):
    
    # list to store prediction
    pred = []
    
    # get image prediction
    def predict_img_sentiment(path):
        try:
            img = cv2.imread(path)
            img = img/255.0
            img = cv2.resize(img, (48,48))
            img = img.reshape(1,48,48,3)
            pred = list(model_j.predict(img)[0])

        except e:
            print("oops!! something went wrong")

        return pred 
    
    # get text sentiment
    def predict_text_sentiment(text):
    
        c = count_vec.transform([text])
        pred = model.predict_proba(c)

        return pred  
    
    
    for i in range(len(df)):
        x = df.loc[i]['image']
        text = df.loc[i]['clean_text']
        
        if df.loc[i]['target_y'] == 1:
            path = test_path + "positive/" + x
        else:
            path = test_path + "negative/" + x
        
        
        img_pred = predict_img_sentiment(path)
        text_pred = predict_text_sentiment(text)
        
        neg_prob = (text_pred[0][0] + img_pred[0])/2
        pos_prob = (text_pred[0][1] + img_pred[1])/2
        
        if neg_prob>pos_prob:
            pred.append(0)
        else:
            pred.append(1)
            
    return np.array(pred)

In [93]:
test_pred = predict(test_df)

In [96]:
test_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [98]:
one = 0
zero = 0
for i in test_pred:
    if i ==1:
        one+=1
    else:
        zero+=1

In [106]:
test_df['target_y'].value_counts().keys()

Int64Index([0, 1], dtype='int64')

In [110]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_pred,test_df['target_y'])

array([[51, 15],
       [33, 68]], dtype=int64)

In [111]:
# calculating accuracy score
acc = (51+68)/(51+15+33+68)

In [113]:
print("accuracy score:",acc)

accuracy score: 0.7125748502994012
